In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from girth import twopl_mml
from scipy.stats import entropy
import matplotlib.pyplot as plt
import seaborn as sns
from constants import SAVE_FOLDER

## Load region-product export data

In [ ]:
trade = pd.read_parquet(f'{SAVE_FOLDER}/cleaned.parquet')
trade.head()

In [ ]:
trade.shape,trade['prod'].nunique()

In [ ]:
R0 = 0.115

In [ ]:
df = trade.assign(llrca = np.log(1+trade.rca/R0)/np.log(1+1/R0))
df['bin'] = df.export>0
df['avgrca_part_p'] = df.llrca*df.pivec_p
df['avgrca_part_m'] = df.llrca*df.pivec_m
df['avgrca_part_c'] = df.llrca*df.pivec_c
df['avgrca_p'] = df.groupby(['year','region'])['avgrca_part_p'].transform(sum)
df['avgrca_m'] = df.groupby(['year','region'])['avgrca_part_m'].transform(sum)
df['avgrca_c'] = df.groupby(['year','region'])['avgrca_part_c'].transform(sum)
df['diversity'] = df.groupby(['year','region'])['binrca'].transform(sum)
df.avgrca_p.describe()

In [ ]:
df['rct_p'] = np.where(df.avgrca_p>0,df.llrca/df.avgrca_p,0)
df['rct_m'] = np.where(df.avgrca_m>0,df.llrca/df.avgrca_m,0)
df['rct_c'] = np.where(df.avgrca_c>0,df.llrca/df.avgrca_c,0)
df['rct_demean_p'] = df.rct_p-df.groupby('prod').rct_p.transform('mean')
df['rct_demean_m'] = df.rct_m-df.groupby('prod').rct_m.transform('mean')
df['rct_demean_c'] = df.rct_c-df.groupby('prod').rct_c.transform('mean')
df['proj_p'] = np.where(df.avgrca_p>0, df.pci_p * df.llrca * df.pivec_p / df.avgrca_p,0)
df['proj_m'] = np.where(df.avgrca_m>0, df.pci_m * df.llrca * df.pivec_m / df.avgrca_m,0)
df['proj_c'] = np.where(df.avgrca_c>0, df.pci_c * df.llrca * df.pivec_c / df.avgrca_c,0)
df['eci_part'] = np.where(df.diversity>0,df.pci * df.binrca/df.diversity,0)
df.head()

In [ ]:
cntryagg = df.groupby(['year','region'])[['avgrca_part_p','avgrca_part_m','avgrca_part_c','proj_p','proj_m','proj_c','eci_part','bin']].sum().reset_index().rename(columns={'avgrca_part_p':'avgrca_p','avgrca_part_m':'avgrca_m','avgrca_part_c':'avgrca_c','eci_part':'eci'})
cntryagg.head()

In [ ]:
cntryagg.year.nunique(),cntryagg.region.nunique()

In [ ]:
cntryagg.shape

In [ ]:
def gen_metric(year):
    ## this is simply a collection of metrics shown in 3_metric_comparision
    ## diversity
    mcpdf = trade[trade.year==year].pivot(index='region',columns='prod',values='binrca').fillna(0)
    mcp = mcpdf.values
    ubiquity = mcp.sum(axis=0)
    diversity = mcp.sum(axis=1)
    mcp = mcp[diversity>0,:]
    #ECI
    kp = mcp.sum(axis=0)
    kc = mcp.sum(axis=1)
    mr = np.diag(1/kc) @ mcp @ np.diag(1/kp) @ mcp.T
    eigvals2,eigvecs2 = np.linalg.eig(mr)
    eigvecs2 = np.real(eigvecs2)
    eci = np.sign(np.corrcoef(kc,eigvecs2[:,1])[0,1])*eigvecs2[:,1]
    ## fitness
    qp = np.ones(mcp.shape[1])
    fc = np.ones(mcp.shape[0])
    for i in range(20):
        fc_t = mcp @ qp
        qp_t = 1/(mcp.T @ (1/fc))
        fc = fc_t / fc_t.mean()
        qp = qp_t / qp_t.mean()
    ## fitness v2
    delta = 0.001
    pp2 = np.ones(mcp.shape[1])/delta
    fc2 = np.ones(mcp.shape[0])*delta
    for i in range(100):
        fc2_tmp = delta*delta + mcp @ (1/pp2)
        pp2_tmp = 1 + mcp.T @ (1/fc2)
        fc2 = fc2_tmp
        pp2 = pp2_tmp
    qp2 = 1/(pp2-1)
    ## genepy
    kp_1 = (np.diag(1/kc) @ mcp).sum(axis=0)
    wcp = np.diag(1/kc) @ mcp @ np.diag(1/kp_1)
    ncc = wcp @ wcp.T
    np.fill_diagonal(ncc, 0)
    eigvals,eigvecs = np.linalg.eig(ncc)
    xc1 = np.absolute(eigvecs[:,0])
    xc2 = eigvecs[:,1]
    lambda1 = eigvals[0]
    lambda2 = eigvals[1]
    genepy = np.square(lambda1*np.square(xc1)+lambda2*np.square(xc2))+2*(lambda1**2*np.square(xc1)+lambda2**2*np.square(xc2))
    ## production ability
    estimates = twopl_mml(mcp.T)
    ## fixed effects
    fedf = trade[(trade.year==year)&(trade.export>0)][['region','prod','export','rca','regionsum','prodsum']].copy()
    fedf['ycp'] = -np.log(-np.log(fedf.rca/(fedf.rca+1)))
    fedf['regionshare'] = fedf.export/fedf.regionsum
    fedf['prodshare'] = fedf.export/fedf.prodsum
    res = smf.ols(formula='ycp ~ region+prod', data=fedf).fit()
    fecoefdf = pd.DataFrame({'fe':res.params[1:]}).reset_index()
    fecoefdf['var'] = fecoefdf['index'].str[-4:-1]
    gamma_c = fedf[['region']].drop_duplicates().merge(fecoefdf[['var','fe']].rename(columns={'var':'region'}),how='left').fillna(0)
    ## entropy
    tmpdf2 = fedf[['region','prod','export']].copy()
    tmpdf2['hc'] = tmpdf2.groupby('region')['export'].transform(entropy)
    tmpdf2['hp'] = tmpdf2.groupby('prod')['export'].transform(entropy)
    tmpdf2['xcp'] = tmpdf2.export*(np.log(233) - tmpdf2.hp)
    tmpdf2['ycp'] = tmpdf2.export*(np.log(235) - tmpdf2.hc)
    tmpdf2['xcpr'] = tmpdf2.xcp/tmpdf2.groupby('region')['xcp'].transform(sum)
    tmpdf2['ycpr'] = tmpdf2.ycp/tmpdf2.groupby('prod')['ycp'].transform(sum)
    for i in range(25):
        tmpdf2['hc'] = tmpdf2.groupby('region')['xcpr'].transform(entropy)
        tmpdf2['hp'] = tmpdf2.groupby('prod')['ycpr'].transform(entropy)
        tmpdf2['xcp'] = tmpdf2.export*(np.log(233) - tmpdf2.hp)
        tmpdf2['ycp'] = tmpdf2.export*(np.log(235) - tmpdf2.hc)
        tmpdf2['xcpr'] = tmpdf2.xcp/tmpdf2.groupby('region')['xcp'].transform(sum)
        tmpdf2['ycpr'] = tmpdf2.ycp/tmpdf2.groupby('prod')['ycp'].transform(sum)
    regiondf2 = tmpdf2[['region','hc']].drop_duplicates().sort_values('region')
    ## collect result
    resdf = pd.DataFrame(
        {
            "fitness_year": fc,
            "fitness_v2": fc2,
            "eci_year": eci,
            "kc": kc,
            "xc1": xc1,
            "xc2": xc2,
            "genepy": genepy,
            "ability": estimates["Ability"],
        },
        index=mcpdf.index[diversity > 0],
    ).reset_index()
    resdf = resdf.merge(gamma_c).merge(regiondf2).merge(cntryagg[(cntryagg.year==year)&(cntryagg.bin>0)].drop(columns=['year']), how='left')
    return resdf

In [ ]:
# Loop over years (~50 min on laptop)
regiondict = dict()
for year in range(1962,2019):
    print(f'processing {year}...')
    try:
        regiondict[f'year{year}'] = gen_metric(year)
    except:
        print(f'{year} has error!')

In [ ]:
region_metricdf = pd.concat([regiondict[f'year{year}'].assign(year=year) for year in range(1962,2019)])
region_metricdf['x1d'] = region_metricdf.xc1 * region_metricdf.kc
region_metricdf['x2divsqrtd'] = region_metricdf.xc2 / np.sqrt(region_metricdf.kc)

In [ ]:
## adjust signs of x2divsqrtd for comparision
signdf = np.sign(region_metricdf.groupby('year')[['x2divsqrtd','eci_year']].corr().unstack().iloc[:,1]).reset_index()
signdf.columns=['year','sign']
region_metricdf = region_metricdf.merge(signdf,how='left')
region_metricdf['x2divsqrtd'] = region_metricdf['x2divsqrtd'] * region_metricdf['sign']

In [ ]:
region_year_corr = region_metricdf.drop(columns=['region']).groupby('year').corr().stack().reset_index()
region_year_corr.columns=['year','metric1','metric2','corrcoef']
region_year_corr.head()

In [ ]:
region_year_corr2 = region_metricdf.drop(columns=['region']).groupby('year').corr(method='spearman').stack().reset_index()
region_year_corr2.columns=['year','metric1','metric2','corrcoef']
region_year_corr2.head()

In [ ]:
region_year_corr.to_csv(f'{SAVE_FOLDER}/region_year_corr.tsv',sep='\t',index=False)
region_year_corr2.to_csv(F'{SAVE_FOLDER}/region_year_rankcorr.tsv',sep='\t',index=False)